# **Jupyter notebook for time series forecasting based on machine learning**
### **01 Description**

This Jupyter notebook is used for traffic flow time series prediction, using the GRU model and the TGCN model (https://ieeexplore.ieee.org/abstract/document/8809901). The experimental data comes from the following related code.

### **02 Import libraries and complete instantiation**

In [ ]:
# Import necessary modules
from utils.data_process import create_dataloader
from instance_parameters import Predict
from models.exp1_gru import GRU
from models.exp1_tgcn import TGCN, load_adjacency_matrix
from utils.predict import predict
from utils.test_and_inspect_fit import test, inspect_model_fit
from utils.train import train
import torch

**The parameters you can adjust** (for example, instance.model = 'GRU')
**Parameter**     | **Value**
-------- | -----
model  | 'TGCN'/'GRU'
input_size  | 40
use_gpu  | False
device  | 0
lr  | 0.001
epochs  | 100
batch_size  | 32
hidden_size  | 48
data_path  | 'dataset/filled_sz_speed.csv'
target  | '92878'
window_size  |  48
pre_len  | 4
shuffle  | True

In [ ]:

# Create an instance of the Predict class
instance = Predict()
instance.model = 'GRU'
instance.input_size = 40
instance.hidden_size = 10
instance.data_path = 'dataset/sz_speed.csv'
instance.target = '90224'
instance.epochs = 100

### **03 Confirm used device（GPU or CPU）**

In [ ]:
if isinstance(instance.device, int) and instance.use_gpu:
    device = torch.device("cuda:" + f'{instance.device}')
else:
    device = torch.device("cpu")
train_loader, test_loader, valid_loader, scaler= create_dataloader(instance, device)
print("Used device:", device)

### 03 Instantiate the model

In [ ]:
try:
    print(f"Start initializing the {instance.model} model")
    if instance.model == 'GRU':
        model = GRU(input_dim = instance.input_size, hidden_size = instance.hidden_size, args = instance)
    elif instance.model == 'TGCN':
        adj = load_adjacency_matrix('dataset/sz_adj.csv')
        model = TGCN(adj = adj, hidden_size = instance.hidden_size ,args = instance)
    print(f"Initialization of the {instance.model} model is complete")
except Exception as e:
    print(f"Initialization of the {instance.model} model failed")
    print(f"Failure Details:{e}")

### 04 Train model

In [ ]:
if instance.train:
    print(f"Start {instance.model} model training")
    train(model, instance, train_loader, scaler)

### 05 test model

In [ ]:
if instance.test:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始{instance.model}模型测试<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    test(model, instance, test_loader, scaler)

### 06 Inspect model fitting results

In [ ]:
if instance.inspect_fit:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始检验{instance.model}模型拟合情况<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    inspect_model_fit(model, instance, train_loader, scaler)

In [ ]:
if instance.predict:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>预测未来{instance.pre_len}条数据<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    predict(model, instance, device, scaler)